In [24]:
import pandas as pd 
import requests
from lxml import etree
import time
import random
import pymongo

In [25]:
df = pd.read_csv("51job_北京_深度学习_列表页.csv") # 读取文件

In [26]:
df.columns 

Index(['详情页链接', '职位名', '公司名', '工作地点', '薪资', '发布时间'], dtype='object')

In [27]:
# 链接mongodb数据库-->默认为host=127.0.0.1 port = 27017
conn = pymongo.MongoClient();
# 创建jobspider数据库
db = conn.jobspider
# 创建qcwy集合
post_info = db.qcwy

### 抓取详情页时出现的两个异常：
    1. 网页声明的编码方式并非其真实编码方式。
    2. 网页数据存放的标签格式不统一，导致某些网页提取不到数据。
#### 解决方法：
    记录出现异常的url，继续下次一次数据抓取。

In [ ]:
# 所要抓取的招聘信息的总数
data_length = df.shape[0]
# 当程序出现异常时用于存放招聘信息得详情页url列表。
errUrl_list = []

In [55]:
# 所要抓取的招聘信息的总数
data_length = df.shape[0]
# 当程序出现异常时用于存放招聘信息得详情页url列表。
errUrl_list = []
for i in range(data_length):
    content=''
    print("--"*20+"开始抓取第"+str(i)+"条招聘信息"+"--"*20)
    
    # 分别取取详情页链接、职位名、公司名、薪资
    detail_page_url = df["详情页链接"][i]
    jobName = df["职位名"][i]
    companyName = df["公司名"][i]
    salary = df["薪资"][i]
    
    # 设置请求头
    headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36'
    }
    #设置休眠时间，防止反爬
    sleep_time = random.choice([3,4,5])+random.random()
    # 发起请求，接收响应
    response = requests.get(detail_page_url,headers=headers)
    try:
        # 按照网页声明的编码方式进行解码
        str_html = response.content.decode(response.apparent_encoding)
    except:
        # 解码失败，将url添加到errUrl_list中
        errUrl_list.append(detail_page_url)
        print(">>"*20+"第"+str(i)+"条招聘信息抓取失败"+"<<"*20)
        print()
        continue
    # 从字符串常量解析HTML文档。
    html = etree.HTML(str_html)
    # 提取信息
    info_length = len(html.xpath('//*[@class="bmsg job_msg inbox"]/p'))
    if info_length == 0:
        # 如果提取不到信息就将url添加到errUrl_list中
        errUrl_list.append(detail_page_url)
        print(">>"*20+"第"+str(i)+"条招聘信息抓取失败"+"<<"*20)
    else:
        for index in range(info_length):
            tag_con = html.xpath('//*[@class="bmsg job_msg inbox"]/p')[index].xpath('string(.)')
            tag_con = tag_con.replace(' ','')
            content =content +tag_con
        data_insert_db ={
                "jobName":jobName,
                "companyName":companyName,
                "salary":salary,
                "content":content
        }
        print(content)
        # 将数据掺入到mongodb中
        post_info.insert_one(data_insert_db)
        print("--"*20+"第"+str(i)+"条招聘信息抓取结束"+"--"*20)
        # 休眠一段时间再抓取下一条数据
        time.sleep(sleep_time)
    print()

----------------------------------------开始抓取第0条招聘信息----------------------------------------


KeyError: '详情页链接'

In [35]:
# 打印errUrl_list的长度
print(len(errUrl_list))

97

In [54]:
# 将出错的url信息写入csv文件中。以备下次分析使用。
df = pd.DataFrame(data = errUrl_list)
df.to_csv("err_url_List.csv")